In [1]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')
import os
import pandas as pd
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import shapefile
from shapely.geometry  import Point, LineString, Polygon, MultiPolygon

from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm
import glob

/home/tserekh/jupnb/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from main_map.models import OrganizationNatClass, House, Metro, MetroStation, PolygonModel, GroundStop, OperSquare
from main_map.models import Organization, OrganizationNatClass, Workplace, House, MetroStation, Metro, GroundStop
from django.core import serializers
from django.utils.safestring import mark_safe

In [3]:
import sys
import json
sys.path.append('/usr/lib/python3/dist-packages')
from pyproj import Proj, transform
def add3857(row):
    xy = transform(PROJ4326, PROJ3857, row['lon'], row['lat'])
    return xy

In [4]:
import pandas as pd

In [5]:
df_polygons = pd.read_csv('/home/tserekh/evil/azbuka_/data/empty_polygons.csv',index_col='Id')

In [6]:
df = pd.read_csv('/home/tserekh/abc_data/oper_data/report1.csv',sep=';',index_col='zid_500')

/home/tserekh/jupnb/lib/python3.5/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
def age_index_to_age(x):
    if x<=1:
        return x*12000
    elif x<=2:
        return 12000 + (x-1)*(46000 - 12000)
    elif x<=3:
        return 46000 + (x-2)*(46000 - 100000)
    else:
        return 100000 + (x-3)*(100000 - 150000)


In [7]:
df['доход'].value_counts(normalize=True)

Средний доход. Более 12000 и до 46000 рублей           0.409301
не определено                                          0.262513
Доход выше среднего. Более 46000 и до 100000 рублей    0.181204
Минимальный доход. От 0 до 12000 рублей                0.090398
Высокий доход. Более 100000 рублей                     0.056584
Name: доход, dtype: float64

In [8]:
df['доход'] = df['доход'].apply(lambda x: x.split('.')[0])

In [9]:
df['возраст'].value_counts(normalize=True)

35-44            0.215627
25-34            0.194194
45-54            0.177393
55-63            0.158447
>=64             0.102595
18-24            0.073119
не определено    0.050573
0-18             0.028052
Name: возраст, dtype: float64

In [10]:
df.rename(
    columns = {
        'тип локации':'if_home',
        'пол':'male',
        'кол-во абонентов':'users',
        'доход':'income',
        'возраст':'age',
    },
    inplace=True


)

In [11]:
df_polygons['lon'] = (df_polygons['0']+df_polygons['2']+df_polygons['4']+df_polygons['6'])/4
df_polygons['lat'] = (df_polygons['1']+df_polygons['3']+df_polygons['5']+df_polygons['7'])/4

In [12]:
PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [13]:
df_polygons['xy'] = df_polygons.apply(add3857, axis = 1)
df_polygons['x'] = df_polygons['xy'].apply(lambda xy:xy[0])
df_polygons['y'] = df_polygons['xy'].apply(lambda xy:xy[1])
df_polygons = df_polygons[['x','y']]

In [14]:
df['age'].value_counts()

35-44            808192
25-34            727857
45-54            664888
55-63            593874
>=64             384536
18-24            274058
не определено    189552
0-18             105140
Name: age, dtype: int64

In [15]:
df['if_home'] = df['if_home']=='дом'

In [16]:
def age_to_int(x):
    dic = {}
    dic['не определено'] = None
    dic['0-18'] = 0
    dic['18-24'] = 1
    dic['25-34'] = 2
    dic['35-44'] = 3
    dic['45-54'] = 4
    dic['55-63'] = 5
    dic['>=64'] = 6
    return dic[x]


In [17]:
def income_to_int(x):
    dic = {}
    dic['не определено'] = None
    dic['Минимальный доход'] = 0
    dic['Средний доход'] = 1
    dic['Доход выше среднего'] = 2
    dic['Высокий доход'] = 3
    return dic[x]

In [18]:
def male_to_bool(x):
    if x=='муж':
        return True
    elif x=='жен':
        return False
    else:
        return None
df['male'] = df['male'].apply(male_to_bool)


In [19]:
set(df['income'])

{'Высокий доход',
 'Доход выше среднего',
 'Минимальный доход',
 'Средний доход',
 'не определено'}

In [20]:
df['income'] = df['income'].apply(income_to_int)

In [21]:
df['income'].value_counts()

1.0    1534099
2.0     679172
0.0     338819
3.0     212081
Name: income, dtype: int64

In [22]:
df = df.join(df_polygons)

In [23]:
df.columns

Index(['if_home', 'male', 'age', 'income', 'users', 'x', 'y'], dtype='object')

In [24]:
df['age'] = df['age'].apply(age_to_int)

In [25]:
df['source_id'] = list(df.index)

In [ ]:
count = OperSquare.objects.count()
if count!=0:
    print('Objects:', count)
    print("don't need to insert")
else:


    features = ['if_home', 'male', 'age', 'income', 'users', 'x', 'y','source_id']

    print(len(df))
    for i, row in tqdm(df[features].iterrows()):
        square = OperSquare(**row.to_dict())
        square.save()

3748097


3685876it [2:16:18, 450.69it/s]